In [1]:
#Wordle Solver
'''
I will tackle this in the following way:
	Get wordle's word list
	Try a word, preferrably one with more vowels (I will manually input words into the browser for now)
	Input the result of the guess with alphabets (G=Green,Y=Yellow,B=Black) to denote positions and errors
	Filter words based on output:
		Keep words with green
		Keep words with same letter as yellow, then remove words with letters in same position
		Remove words with black
	Repeat process until I either get the word or I run out of tries
'''

"\nI will tackle this in the following way:\n\tGet wordle's word list\n\tTry a word, preferrably one with more vowels (I will manually input words into the browser for now)\n\tInput the result of the guess with alphabets (G=Green,Y=Yellow,B=Black) to denote positions and errors\n\tFilter words based on output:\n\t\tKeep words with green\n\t\tKeep words with same letter as yellow, then remove words with letters in same position\n\t\tRemove words with black\n\tRepeat process until I either get the word or I run out of tries\n"

In [2]:
import random

In [3]:
def CreateWordList(WordList):
	with open('WordList.txt','r',encoding='UTF-8') as file:
		while line := file.readline():
			WordList.append(line.rstrip())

def MakeGuess(status,WordList,guess):
	if len(status)==0:
		print(f"Guess: {(guess := random.choice(WordList))}")
		return guess
	else:
		for i in range(len(status)):
			if status[i] == 'G':
				j = 0
				while (j<len(WordList)):
					if WordList[j][i] != guess[i]:
						WordList.pop(j)
					else:
						j += 1
			elif status[i] == 'Y':
				j = 0
				while (j<len(WordList)):
					if guess[i] not in WordList[j]:
						WordList.pop(j)
					else:
						j += 1
				j = 0
				while (j<len(WordList)):
					if WordList[j][i] == guess[i]:
						WordList.pop(j)
					else:
						j += 1
			elif status[i] == 'B':
				j = 0
				while (j<len(WordList)):
					if ((guess[i] in WordList[j]) and WordList[j].count(guess[i]) >= guess.count(guess[i])):
						#print(f"Letter: {guess[i]}\nOccurence in guess: {guess.count(guess[i])}\nOccurence in word: {WordList[j].count(guess[i])}\nRemoving word: {WordList[j]}")
						WordList.pop(j)
					else:
						j += 1
			else:
				print("Wrong input lol, I need to automate this")
		print(f"Guess: {(guess := random.choice(WordList))}")
		return guess
						

In [ ]:
WordList = []
status = ""
guess = ""
tries = 1

CreateWordList(WordList)

while (tries <= 6):
    guess = MakeGuess(status,WordList,guess)
    status = input("Enter status: ")
    if status == "GGGGG":
        break
    tries += 1

Guess: kyats
Guess: amnio
Guess: arrow
Guess: abhor
